In [1]:
param_x_api_key = '2BeBXg69osbuG45Rjtv1wLEnHL3nuT3WPYUZc8Nd3ZL6LpSuVzfA'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part_number.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                data = '''{"getArticles":
                {"articleCountry":"DE",
                 "provider":40000,
                 "lang":"en",
                 "searchQuery":"''' + input_.loc[a, 'Part_Number'].strip() + '''",
                 "searchMatchType":"prefix_or_suffix",
                 "searchType":10,
                 "page":1,
                 "perPage":100,
                 "sort":[{"field":"score",
                          "direction":"desc"},
                         {"field":"mfrName",
                          "direction":"asc"},
                         {"field":"linkageSortNum",
                          "direction":"asc"}],
                 "filterQueries":["(dataSupplierId NOT IN (4978,4982))"],
                 "dataSupplierIds":[],
                 "genericArticleIds":[],
                 "includeAll":false,
                 "includeLinkages":true,
                 "linkagesPerPage":100,
                 "includeGenericArticles":true,
                 "includeArticleCriteria":true,
                 "includeMisc":true,
                 "includeImages":true,
                 "includePDFs":true,
                 "includeLinks":true,
                 "includeArticleText":true,
                 "includeOEMNumbers":true,
                 "includeReplacedByArticles":true,
                 "includeReplacesArticles":true,
                 "includeComparableNumbers":true,
                 "includeGTINs":true,
                 "includeTradeNumbers":true,
                 "includePrices":false,
                 "includePartsListArticles":false,
                 "includeAccessoryArticles":false,
                 "includeArticleLogisticsCriteria":true,
                 "includeDataSupplierFacets":true,
                 "includeGenericArticleFacets":true,
                 "includeCriteriaFacets":false}}'''

                headers = UA.get_User_Agent_Requests()
                headers['X-Api-Key'] = param_x_api_key

                resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                                     data=data,
                                     headers=headers,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                list_part = json_data['articles']
                
                # = = = = = = = = = = = = = = =
                
                index_part = -1
                for i in range(len(list_part)):
                    if list_part[i]['articleNumber'].strip() == input_.loc[a, 'Part_Number'].strip():
                        index_part = i
                        break
                
                # = = = = = = = = = = = = = = =
                
                if index_part == -1:
                    continue

                # = = = = = = = = = = = = = = =
                
                oe = ';'.join(sorted(list(set(oe['articleNumber'].strip().replace(' ', '') for oe in list_part[index_part]['oemNumbers']))))

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                                         'OE': oe}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Part_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()

output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./oe-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./oe_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1051

[ok] - 12-161758
[尝试次数：1] - [剩余数量：1031] - [当前时间：21:43:13]

[ok] - 14-216033
[尝试次数：1] - [剩余数量：1030] - [当前时间：21:43:13]

[ok] - 14-236040
[尝试次数：1] - [剩余数量：1029] - [当前时间：21:43:13]

[ok] - 14-236033
[尝试次数：1] - [剩余数量：1028] - [当前时间：21:43:14]

[ok] - 12-211451
[尝试次数：1] - [剩余数量：1027] - [当前时间：21:43:14]

[ok] - 14-216103
[尝试次数：1] - [剩余数量：1026] - [当前时间：21:43:14]

[ok] - 12-080601
[尝试次数：1] - [剩余数量：1025] - [当前时间：21:43:14]

[ok] - 12-101442
[尝试次数：1] - [剩余数量：1024] - [当前时间：21:43:14]

[ok] - 12-211796
[尝试次数：1] - [剩余数量：1023] - [当前时间：21:43:14]

[ok] - 12-211747
[尝试次数：1] - [剩余数量：1022] - [当前时间：21:43:14]

[ok] - 14-216090
[尝试次数：1] - [剩余数量：1021] - [当前时间：21:43:14]

[ok] - 12-090225
[尝试次数：1] - [剩余数量：1020] - [当前时间：21:43:14]

[ok] - 12-212022
[尝试次数：1] - [剩余数量：1019] - [当前时间：21:43:14]

[ok] - 12-211449
[尝试次数：1] - [剩余数量：1018] - [当前时间：21:43:14]

[ok] - 14-216075
[尝试次数：1] - [剩余数量：1017] - [当前时间：21:43:14]

[ok] - 14-216036
[尝试次数：1] - [剩余数量：1016] - [当前时间：21:43:15]

[ok] - 14-236034
[尝试次数：1] - [剩余数量：1015] - [当前时

[ok] - 12-226003
[尝试次数：1] - [剩余数量：889] - [当前时间：21:43:38]

[ok] - 12-231396
[尝试次数：1] - [剩余数量：888] - [当前时间：21:43:38]

[ok] - 12-050481
[尝试次数：1] - [剩余数量：887] - [当前时间：21:43:38]

[ok] - 12-161477
[尝试次数：1] - [剩余数量：886] - [当前时间：21:43:38]

[ok] - 12-211690
[尝试次数：1] - [剩余数量：885] - [当前时间：21:43:39]

[ok] - 12-050451
[尝试次数：1] - [剩余数量：884] - [当前时间：21:43:39]

[ok] - 12-011714
[尝试次数：1] - [剩余数量：883] - [当前时间：21:43:40]

[ok] - 12-040885
[尝试次数：1] - [剩余数量：882] - [当前时间：21:43:40]

[ok] - 12-080650
[尝试次数：1] - [剩余数量：881] - [当前时间：21:43:40]

[ok] - 14-216101
[尝试次数：6] - [剩余数量：880] - [当前时间：21:43:41]

[ok] - 12-120675
[尝试次数：1] - [剩余数量：879] - [当前时间：21:43:41]

[ok] - 12-300737
[尝试次数：2] - [剩余数量：878] - [当前时间：21:43:41]

[ok] - 12-141454
[尝试次数：1] - [剩余数量：877] - [当前时间：21:43:41]

[ok] - 12-120432
[尝试次数：1] - [剩余数量：876] - [当前时间：21:43:41]

[ok] - 14-066168
[尝试次数：2] - [剩余数量：875] - [当前时间：21:43:42]

[ok] - 12-221680
[尝试次数：1] - [剩余数量：874] - [当前时间：21:43:42]

[ok] - 12-090222
[尝试次数：1] - [剩余数量：873] - [当前时间：21:43:42]

[ok] - 14-2260

[ok] - 14-216130
[尝试次数：1] - [剩余数量：747] - [当前时间：21:44:37]

[ok] - 12-080700
[尝试次数：1] - [剩余数量：746] - [当前时间：21:44:37]

[ok] - 12-211727
[尝试次数：1] - [剩余数量：745] - [当前时间：21:44:37]

[ok] - 12-080693
[尝试次数：1] - [剩余数量：744] - [当前时间：21:44:37]

[ok] - 12-141791
[尝试次数：1] - [剩余数量：743] - [当前时间：21:44:38]

[ok] - 12-120673
[尝试次数：1] - [剩余数量：742] - [当前时间：21:44:38]

[ok] - 12-050428
[尝试次数：1] - [剩余数量：741] - [当前时间：21:44:39]

[ok] - 12-080602
[尝试次数：1] - [剩余数量：740] - [当前时间：21:44:39]

[ok] - 12-120669
[尝试次数：1] - [剩余数量：739] - [当前时间：21:44:40]

[ok] - 12-080658
[尝试次数：1] - [剩余数量：738] - [当前时间：21:44:41]

[ok] - 12-211322
[尝试次数：1] - [剩余数量：737] - [当前时间：21:44:42]

[ok] - 12-120434
[尝试次数：1] - [剩余数量：736] - [当前时间：21:44:44]

[ok] - 12-040863
[尝试次数：1] - [剩余数量：735] - [当前时间：21:44:44]

[ok] - 12-040843
[尝试次数：1] - [剩余数量：734] - [当前时间：21:44:45]

[ok] - 12-212005
[尝试次数：3] - [剩余数量：733] - [当前时间：21:44:45]

[ok] - 12-211694
[尝试次数：7] - [剩余数量：732] - [当前时间：21:44:45]

[ok] - 14-216118
[尝试次数：1] - [剩余数量：731] - [当前时间：21:44:45]

[ok] - 12-0017

[ok] - 12-060192
[尝试次数：1] - [剩余数量：604] - [当前时间：21:45:50]

[ok] - 12-211994
[尝试次数：1] - [剩余数量：603] - [当前时间：21:45:50]

[ok] - 12-050427
[尝试次数：1] - [剩余数量：602] - [当前时间：21:45:51]

[ok] - 12-341956
[尝试次数：1] - [剩余数量：601] - [当前时间：21:45:51]

[ok] - 12-291991
[尝试次数：1] - [剩余数量：600] - [当前时间：21:45:51]

[ok] - 14-356067
[尝试次数：1] - [剩余数量：599] - [当前时间：21:45:52]

[ok] - 12-050432
[尝试次数：1] - [剩余数量：598] - [当前时间：21:45:52]

[ok] - 12-221688
[尝试次数：1] - [剩余数量：597] - [当前时间：21:45:52]

[ok] - 14-356084
[尝试次数：1] - [剩余数量：596] - [当前时间：21:45:53]

[ok] - 14-216074
[尝试次数：1] - [剩余数量：595] - [当前时间：21:45:53]

[ok] - 12-080672
[尝试次数：2] - [剩余数量：594] - [当前时间：21:45:53]

[ok] - 12-151980
[尝试次数：1] - [剩余数量：593] - [当前时间：21:45:54]

[ok] - 12-231392
[尝试次数：1] - [剩余数量：592] - [当前时间：21:45:54]

[ok] - 12-120435
[尝试次数：1] - [剩余数量：591] - [当前时间：21:45:55]

[ok] - 12-011367
[尝试次数：1] - [剩余数量：590] - [当前时间：21:45:55]

[ok] - 14-356081
[尝试次数：1] - [剩余数量：589] - [当前时间：21:45:55]

[ok] - 14-016052
[尝试次数：1] - [剩余数量：588] - [当前时间：21:45:56]

[ok] - 12-0806

[ok] - 12-351952
[尝试次数：2] - [剩余数量：461] - [当前时间：21:46:32]

[ok] - 12-251662
[尝试次数：1] - [剩余数量：460] - [当前时间：21:46:32]

[ok] - 14-356061
[尝试次数：1] - [剩余数量：459] - [当前时间：21:46:32]

[ok] - 14-146062
[尝试次数：2] - [剩余数量：458] - [当前时间：21:46:33]

[ok] - 12-011628
[尝试次数：6] - [剩余数量：457] - [当前时间：21:46:33]

[ok] - 12-080642
[尝试次数：3] - [剩余数量：456] - [当前时间：21:46:34]

[ok] - 12-341954
[尝试次数：2] - [剩余数量：455] - [当前时间：21:46:34]

[ok] - 12-090221
[尝试次数：2] - [剩余数量：454] - [当前时间：21:46:34]

[ok] - 12-291372
[尝试次数：1] - [剩余数量：453] - [当前时间：21:46:35]

[ok] - 12-090245
[尝试次数：1] - [剩余数量：452] - [当前时间：21:46:35]

[ok] - 12-040867
[尝试次数：3] - [剩余数量：451] - [当前时间：21:46:36]

[ok] - 14-346061
[尝试次数：6] - [剩余数量：450] - [当前时间：21:46:36]

[ok] - 12-080680
[尝试次数：5] - [剩余数量：449] - [当前时间：21:46:36]

[ok] - 12-060141
[尝试次数：3] - [剩余数量：448] - [当前时间：21:46:37]

[ok] - 12-211681
[尝试次数：4] - [剩余数量：447] - [当前时间：21:46:37]

[ok] - 12-271699
[尝试次数：2] - [剩余数量：446] - [当前时间：21:46:37]

[ok] - 12-251319
[尝试次数：2] - [剩余数量：445] - [当前时间：21:46:37]

[ok] - 12-0806

[ok] - 12-060106
[尝试次数：1] - [剩余数量：319] - [当前时间：21:47:25]

[ok] - 12-231974
[尝试次数：1] - [剩余数量：318] - [当前时间：21:47:25]

[ok] - 12-221778
[尝试次数：1] - [剩余数量：317] - [当前时间：21:47:26]

[ok] - 12-271613
[尝试次数：1] - [剩余数量：316] - [当前时间：21:47:27]

[ok] - 12-300751
[尝试次数：1] - [剩余数量：315] - [当前时间：21:47:27]

[ok] - 12-251656
[尝试次数：1] - [剩余数量：314] - [当前时间：21:47:28]

[ok] - 12-040883
[尝试次数：1] - [剩余数量：313] - [当前时间：21:47:28]

[ok] - 12-271611
[尝试次数：1] - [剩余数量：312] - [当前时间：21:47:29]

[ok] - 12-351436
[尝试次数：1] - [剩余数量：311] - [当前时间：21:47:29]

[ok] - 12-080621
[尝试次数：1] - [剩余数量：310] - [当前时间：21:47:30]

[ok] - 12-211989
[尝试次数：1] - [剩余数量：309] - [当前时间：21:47:30]

[ok] - 12-141459
[尝试次数：1] - [剩余数量：308] - [当前时间：21:47:31]

[ok] - 12-060135
[尝试次数：1] - [剩余数量：307] - [当前时间：21:47:31]

[ok] - 12-292004
[尝试次数：1] - [剩余数量：306] - [当前时间：21:47:32]

[ok] - 12-291373
[尝试次数：1] - [剩余数量：305] - [当前时间：21:47:33]

[ok] - 12-341953
[尝试次数：2] - [剩余数量：304] - [当前时间：21:47:34]

[ok] - 12-351999
[尝试次数：1] - [剩余数量：303] - [当前时间：21:47:34]

[ok] - 12-2716

[ok] - 12-011997
[尝试次数：1] - [剩余数量：177] - [当前时间：21:48:29]

[ok] - 12-141466
[尝试次数：2] - [剩余数量：176] - [当前时间：21:48:29]

[ok] - 12-011989
[尝试次数：1] - [剩余数量：175] - [当前时间：21:48:30]

[ok] - 12-271800
[尝试次数：1] - [剩余数量：174] - [当前时间：21:48:30]

[ok] - 12-351766
[尝试次数：1] - [剩余数量：173] - [当前时间：21:48:30]

[ok] - 12-351987
[尝试次数：1] - [剩余数量：172] - [当前时间：21:48:31]

[ok] - 12-011984
[尝试次数：1] - [剩余数量：171] - [当前时间：21:48:31]

[ok] - 12-051713
[尝试次数：1] - [剩余数量：170] - [当前时间：21:48:31]

[ok] - 12-271795
[尝试次数：1] - [剩余数量：169] - [当前时间：21:48:31]

[ok] - 12-090252
[尝试次数：1] - [剩余数量：168] - [当前时间：21:48:31]

[ok] - 12-040865
[尝试次数：1] - [剩余数量：167] - [当前时间：21:48:32]

[ok] - 12-080638
[尝试次数：1] - [剩余数量：166] - [当前时间：21:48:32]

[ok] - 12-080704
[尝试次数：1] - [剩余数量：165] - [当前时间：21:48:32]

[ok] - 12-080701
[尝试次数：1] - [剩余数量：164] - [当前时间：21:48:32]

[ok] - 12-050484
[尝试次数：1] - [剩余数量：163] - [当前时间：21:48:33]

[ok] - 12-050483
[尝试次数：1] - [剩余数量：162] - [当前时间：21:48:33]

[ok] - 12-292006
[尝试次数：1] - [剩余数量：161] - [当前时间：21:48:33]

[ok] - 12-0119

[ok] - 14-306752
[尝试次数：1] - [剩余数量：34] - [当前时间：21:49:22]

[ok] - 12-260314
[尝试次数：1] - [剩余数量：33] - [当前时间：21:49:23]

[ok] - 12-120449
[尝试次数：1] - [剩余数量：32] - [当前时间：21:49:23]

[ok] - 12-040849
[尝试次数：1] - [剩余数量：31] - [当前时间：21:49:24]

[ok] - 12-080681
[尝试次数：1] - [剩余数量：30] - [当前时间：21:49:24]

[ok] - 12-060170
[尝试次数：1] - [剩余数量：29] - [当前时间：21:49:24]

[ok] - 12-090273
[尝试次数：1] - [剩余数量：28] - [当前时间：21:49:24]

[ok] - 12-050489
[尝试次数：1] - [剩余数量：27] - [当前时间：21:49:25]

[ok] - 12-050463
[尝试次数：1] - [剩余数量：26] - [当前时间：21:49:25]

[ok] - 12-161471
[尝试次数：1] - [剩余数量：25] - [当前时间：21:49:25]

[ok] - 12-310592
[尝试次数：1] - [剩余数量：24] - [当前时间：21:49:26]

[ok] - 12-221387
[尝试次数：1] - [剩余数量：23] - [当前时间：21:49:26]

[ok] - 12-050480
[尝试次数：1] - [剩余数量：22] - [当前时间：21:49:27]

[ok] - 12-271402
[尝试次数：1] - [剩余数量：21] - [当前时间：21:49:27]

[ok] - 12-251716
[尝试次数：1] - [剩余数量：20] - [当前时间：21:49:28]

[ok] - 12-080644
[尝试次数：1] - [剩余数量：19] - [当前时间：21:49:28]

[ok] - 12-300729
[尝试次数：1] - [剩余数量：18] - [当前时间：21:49:29]

[ok] - 12-070961
[尝试次数：1] - [剩余

KeyboardInterrupt
2024-04-25T13:53:10Z


KeyboardInterrupt: 